# Homework 5: Project Report
This homework is very similar to homework 5, except now you've hopefully finished your project! You should fill in each of the sections below based on your results. Note that each section has changed from homework 5, so don't just copy paste, the questions are different!

Only one member of the group should submit this assignment, however, please provide the names of each member of the group here:

**Group Members:**
*   Jason Cui
*   James Gan
*   Shine Lin

## Task 1: Problem Description
### *Describe the functionality of your finished product. Try to explain it as if you were writing for a potential customer.*

The Smart Ping Pong Paddle is able to use real-time swing detection, swing spectrum extraction, ball-hit detection and ball-hit quality scoring. These analytical features give you a basic understanding of your skill level and improvement for ping pong. We offer some functions on a stable release, with experimental functions for reference that my have some bugs. Furthermore, our experimental feature allows you to know the quality of your swings without the need to actually serve and hit balls, making it easier to practice without an expensive needing an expensive robot to launch balls at you. This feature allows you to practice your swing as you begin your ping pong journey, saving you from the laborious work of repeatedly hitting and collecting balls- you don't even need a ping pong table!
    
### *Did anything about your project goals change from the beginning of the quarter? What caused the change?*

We originally planned to classify between good/bad players, to classify different swings, and to classify additional variables around what makes a player good. However, it was difficult to attain well-labeled data for this (we had various forms of noise from our sensors that would change whenever we modified our device), and our project ran out of budget well in advance of being able to test out additional methods to get this sort of data without an additional device (such as a sepreate camera). As a result, we changed out project goal from being an over-arching device capable of improving any factor of a player's behavior, to simply providing a device that would allow beginners to improve and track their ping pong skills.

### *If you had more time, what would you do differently?*

If we had more time, would separate our process into additional, smaller steps, to allow us to collect data and train it with waterfall methodology, so that we could ensure a hardware freeze before working on the complex software aspects. Furthermore, that would also allow us to more appropriately distribute work between team members. In this way, we would hope to be able to develop numerous hardware prototypes and develop models for each of them to determine which setups worked best for our end results.
  

  

## Task 2: Dataset Description

### *How did you collect your dataset?*

We collected our data in three different ways:

1. <b>Feasibility Test</b> Initially, we attached a cell phone to a ping pong paddle and used an app to collect acceleration and gyroscope data. The user being monitored swung the paddle repeatedly, as if he were serving a ball.
  
2. <b>FT - Advanced</b> As we started to develop machine learning models and other algorithms, we continued to use this device configuration but monitored additional users and had them either swing the paddle or to walk around intermittently.
  
3. <b>Milestone 1-3</b> Once we had our hardware device created, we began to use it to collect swing-related data from an Inertial Measurement Unit (MPU-6050) and to collect hit-related data using several piezoelectric discs. We asked several testers to repeatedly serve an actual ball, allowing natural practice (such as picking up balls, walking around, and taking breaks).

For our third way, we had various hardware iterations that affected the scaling and noise for each dataset collected.
  
### *What features did your dataset have?*

Our dataset had three primary feature categories:

**Raw input features**: IMU acceleration/IMU angular velocity ($F^6$), piezoelectric disk reading which indicates vibration strength ($N^3$)

**Manually labelled features**: Player Skill/Quality ($B$), Player ID ($N$)

**Engineering features**: Ball Hitting Event ($N$), Ball Hitting Temporal Dispersity (BHTD)/Ball Hitting Spatial Dispersity (BHSD)/Ball Hitting Strength (BHS) ($F^3\times N$), Top-K Signal In Frequency Domain ($F\times N$), Hit Location.


### *What were the labels?*

Some preprocessed features themselves are used as labels, such as the piezometer readings; other labels are manually annotated, such as player skill. Some tasks do not have labels.
    
For swing and hit detection, there were no labels. However, technically speaking, to fine-tune and verify the algorithms, we did manually visualize these features and counted swings and hits. The extra information generated by manually counting swings and hits is implicitly encoded into our model, thus could be treated as labels.
    
For hit quality prediction, labels are the aforementioned float-type engineering features including BHTD, BHSD, and BHS.

### *How many samples did you collect?*

<b>Feasibility Test</b> At least 5 valid files (tests) were collected, each consisting of 20+ swings and hits.
   
<b>Milestone 1-3</b> At least 13 valid files (tests) were collected, of which: 
* 10 of these tests consists of 120+ swings and hits
* 2 of these tests consists of 0 swings or hits for zero offset detection purposes 
* 6 of these tests (with valid swings and hits) are well annotated with exact number of swings and hits, and the names of the testers to distinguish different swing/hit patterns among different testers. 

### *What is your final opinion of your dataset? If you could do it again, what would you change?*

In summary, our dataset has 20 groups of training data from 20 different individuals, in which each person swung the paddle at least 20 times. In our opinion, this dataset is large enough for the basic model to learn to provide a reasonable estimation. Although the total number of samples in the training dataset could be higher, we collected as many features as possible in the process, as advised from our class instructor. 

If we could do it all again, we would make sure that the data from sensors is easy to collect and process. As mentioned previously, our team struggled with piezos data collecting, so we would rather focus on the data processing from the inertial measurement unit. This would give us more time to proceed further in the machine learning process and retrieve deeper insights.

## Task 3: Model Design
### *What type of architecture did you use? Was it the same as you expected? If not, why?*
We used dynamic single layer RNN model with zero state value to train our dataset. We have only one layer in our RNN model with 32 hidden units. It did not fully meet our expectations but it is still useful. We can decrease our regression loss, shown in the below graphs, however, the testing accuracy can be higher. We believe we can do more things with our 70% testing accuracy model. One of the unexpected thing will be explained more in the below paragraph.
  
  **TODO** Same w/ HW4 for the "experimental feature"
  
 <b>Hit Location Detection</b> 
We initially tried to solve for the intersection of three circles to obtain the location of where the ball hits the paddle. These circles have the same location with the deployed three piezo discs, and their diameters are linear to each reading of the sensors. It did not work too well because:

1.   A single intersection usually does not exist, and
2.   Strong hits too close to one sensor could result in a far-away location that is way off the paddle.

We re-implemented the hit location detection algorithm by introducing "weighted centroid," which calculates the centroid of the 3 piezo discs with weights linear to the sensor readings. This worked well (theoretically) if the hit is inside the triangle formed by the 3 discs; otherwise, it is not able to locate. We eventually dropped this feature due to the fact that sensor readings themselves are unreliable.
    
<b>Swing Detection/Counting</b> We counted the number of swings in a given series of data (length unlimited if thresholds are given; otherwise a minimum length is required to ensure algorithm quality) by:

1.   Finding the local maxima along each dimension of the input features,
2.   Auto-clustering these maxima with a given maximum clustering temporal window along each dimension, and
3.   Taking the median of number of clusters of all dimensions.

If a threshold is not given (as is the case when our product "meets" a new player for the first time), we require the player to keep swinging for 10+ seconds. After this, our algorithms learns appropriate thresholds for the player by taking 0.6 times the standard deviation along each dimension. This requires no zero-offseting of any dimension.
    
<b>Hit Detection/Counting</b> A similar mechanism is used for hit detection except that a hard-coded filter is applied to piezo disc readings before they are fed to auto-clustering steps. The maximum clustering window for hit detection is smaller than that used in swing detection.
    
<b>Swing Spectrum</b> Fast Fourier Transform (FFT) is used for extracting the spectrum for swings. Some tweaks include: 

1. Shifting the data first to align its mean to zero before feeding the data to FFT,
2. Drawing a partition window to the temporal data if applicable so that we could handle a mixture of "swings & walks" better, 
3. Only taking some top 10% signals in the frequency domain to generate the spectrum, and 
4. Converting frequency domain to period domain so that the spectrum has a reasonable "length" along the x-axis (noises are usually of randomly-high frequency which have an extremely short period).
    
<b>Hit Quality Prediction/Regression</b> Depending on the deployment type of the machine learning task, we need to consider the supported operations and architectures by TF Lite as we design and train the model.

> **TF Version**

> Input -> SimpleRNN -> Dense -> BatchNorm -> Dropout -> BatchNorm -> Dropout -> Dense (Output)

> **TF Lite Version**

> Input -> Conv1D -> Flatten -> Dense -> BatchNorm -> Dropout -> Dense -> BatchNorm -> Dropout -> Dense (Output)

Since TF Lite does not support RNN nor ELU activations, the RNN has been replaced by Conv1D and ELU has been replaced by ReLU instead.




### *Show the structure of your architecture (graphic here)*

For Hit Quality Prediction/Regression Only:

<img src="https://i.imgur.com/PnHIH5O.jpg"></img>
    
Total params: 1,360
    
Trainable params: 1,312
    
Non-trainable params: 48
  
  


## Task 4: Training
### * What was your final training and test accuracy? How many epochs did it take to reach that?*

Results for the regression after 1024 epochs:
   
<b>Training Set</b>
* Mean Error 0.0496
* 25th Quartile: 0.0184
* 50th Quartile: 0.0338
* 75th Quartile: 0.0659
      
<b>Test Set </b>
* Mean Error 0.1631
* 25th Quartile: 0.0658
* 50th Quartile: 0.1609
* 75th Quartile: 0.2441

Note that actual value roughly range from 0-1, while the predicted value exactly range from 0-1.
    
<b>Loss Along Epochs</b>

<img src="https://github.com/gixhswlab1ppm/ppm/blob/master/insights/Figure_34.png?raw=true"/>
  
<b>Training Set - Error Visualization (order rearranged)</b>

<img src="https://github.com/gixhswlab1ppm/ppm/blob/master/insights/Figure_40.png?raw=true"/>
  
<b>Test Set - Error Visualization 1 (order rearranged)</b>

<img src="https://github.com/gixhswlab1ppm/ppm/blob/master/insights/Figure_42.png?raw=true"/>
  
<b>Test Set - Error Visualization 2 (order rearranged)</b>

<img src="https://github.com/gixhswlab1ppm/ppm/blob/master/insights/Figure_41.png?raw=true"/>
  
### *Did you encounter any unexpected issues during training?*

We were struggling with the parameter tuning process in our training model. For example, when the dropout values are not zero, the model learnt less and was easy to overfit. So our current solution is to set the dropout values to zero, keeping every neuron in our model for a better training result. We think the reason for this is that our dataset is not large enough for the model to actually implement the dropout method. If we have more data, we may not suffer from this same issue in the future.

## Task 5: Deployment

**TODO**

### *Describe the method used to fun your model, how did it fit into the rest of your system?*


### *Are you happy with how the project turned out? Why or why not?*
We think the result now is good enough that our model is learning "something" from our dataset and the regression loss can decrease. This means that we can classify players into  at least two classes (good player and bad player) and meet our initial goal set in the beginning of the lab: helping beginners to learn how to play ping pong. From this perspective, we are happy with the result. But, we also believe that we can let our model do more tasks when it comes to teaching the beginners. For example, adding the tracking lines for the players' swinging feature for the beginners to compare their tracking lines to those who play really well.